In [1]:
# Math Operations
import numpy as np
from math import pi

# Data Preprocessing
import pandas as pd
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
!pip install soccerplots
import soccerplots
from soccerplots.radar_chart import Radar   # for custom radar visuals

# Machine Learning
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Print Statement
print('All necessary libraries imported.')

All necessary libraries imported.


In [2]:
leeds_s1 = pd.read_csv("datasets\cleanedbasic\leeds20-21.csv")
leeds_s2 = pd.read_csv("datasets\cleanedbasic\leeds21-22.csv")
# hudds
hudds_s1 = pd.read_csv("datasets\cleanedbasic\hudds17-18.csv")
hudds_s2 = pd.read_csv("datasets\cleanedbasic\hudds18-19.csv")
# brighton
bri_s1 = pd.read_csv("datasets/cleanedbasic/brighton17-18.csv")
bri_s2 = pd.read_csv("datasets/cleanedbasic/brighton18-19.csv")
# newcastle
new_s1 = pd.read_csv("datasets/cleanedbasic/newcastle17-18.csv")
new_s2 = pd.read_csv("datasets/cleanedbasic/newcastle18-19.csv")
# sheff utd
sheff_s1 = pd.read_csv("datasets/cleanedbasic/sheffutd19-20.csv")
sheff_s2 = pd.read_csv("datasets/cleanedbasic/sheffutd20-21.csv")
# aston villa
villa_s1 = pd.read_csv("datasets/cleanedbasic/villa19-20.csv")
villa_s2 = pd.read_csv("datasets/cleanedbasic/villa20-21.csv")
# wolves
wolves_s1 = pd.read_csv("datasets/cleanedbasic/wolves18-19.csv")
wolves_s2 = pd.read_csv("datasets/cleanedbasic/wolves19-20.csv")

In [3]:
def find_club_files(pathval, team):
    path = pathval
    extension = 'csv'
    os.chdir(path)
    result = glob.glob(team + '*.{}'.format(extension))
    return result

In [4]:
def selected_season_data(yr, team, values):
    values = find_club_files('C:/Users/user/Desktop/xg-data-analysis/datasets/cleanedAdvanced', team)
    x = []
    for i in range(len(values)):
        if yr in values[i]:
            x.append(values[i])
    return x

In [5]:
selected_season_data("2021", "leeds", find_club_files('C:/Users/user/Desktop/xg-data-analysis/datasets/cleanedAdvanced', "leeds"))

['leeds-adkeeper2021-cleaned.csv',
 'leeds-defensive2021-cleaned.csv',
 'leeds-keeper2021-cleaned.csv',
 'leeds-miscell2021-cleaned.csv',
 'leeds-pass2021-cleaned.csv',
 'leeds-playtime2021-cleaned.csv',
 'leeds-shoot2021-cleaned.csv',
 'leeds-shootcreate2021-cleaned.csv']

In [6]:
leeds_s1.columns

Index(['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Per90Gls', 'Per90Ast',
       'Per90G+A', 'Per90G-PK', 'Per90G+A-PK', 'xG', 'npxG', 'xA', 'npxG+xAG',
       'Per90xG', 'Per90xA', 'Per90xG+xA', 'Per90npxG', 'Per90npxG+xAG'],
      dtype='object')

In [7]:
leeds_s1['Pos'].value_counts()

DF      7
MF      4
MFFW    4
DFMF    3
GK      2
MFDF    2
FW      1
Name: Pos, dtype: int64

In [8]:
def join_all_columns(team, yr, df, pos):
    data = find_club_files('C:/Users/user/Desktop/xg-data-analysis/datasets/cleanedAdvanced', team)
    files = selected_season_data(yr, team, data)
    df1 = df.loc[df['Pos'] != pos]
    final = []
    for i in range(len(files)):
        if "keeper" not in files[i]:
            f = pd.read_csv(files[i])
            f = f.loc[f['Pos'] != pos]
            final.append(f)
    return final

In [9]:
vals = join_all_columns("Leeds", "2021", leeds_s1, "GK")

In [10]:
from functools import reduce
leeds_s1_all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), vals)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/633160844.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  leeds_s1_all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), vals)


In [11]:
leeds_s1_all.head()

,Player,Nation,Pos,Age,90s_x,Tkl,TklWon_x,TacklesDef3rd,TacklesMid3rd,TacklesAtt3rd,...,SCAFld,SCADef,GCA,GCA90,GCAPassLive,GCAPassDead,GCADrib,GCASh,GCAFld,GCADef
0,Stuart Dallas,nir NIR,MFDF,29.0,37.9,74,45,46,22,6,...,1,2,7,0.18,7,0,0,0,0,0
1,Luke Ayling,eng ENG,DF,28.0,37.8,108,61,69,30,9,...,3,0,4,0.11,4,0,0,0,0,0
2,Patrick Bamford,eng ENG,FW,26.0,33.9,21,9,2,11,8,...,4,0,16,0.47,9,0,2,2,3,0
3,Jack Harrison,eng ENG,MFFW,23.0,31.6,63,38,20,30,13,...,1,0,15,0.47,12,1,2,0,0,0
4,Ezgjan Alioski,mk MKD,DFMF,28.0,27.3,74,31,38,28,8,...,3,1,5,0.18,3,1,0,1,0,0


In [12]:
leeds_s1_all.columns

Index(['Player', 'Nation', 'Pos', 'Age', '90s_x', 'Tkl', 'TklWon_x',
       'TacklesDef3rd', 'TacklesMid3rd', 'TacklesAtt3rd',
       ...
       'SCAFld', 'SCADef', 'GCA', 'GCA90', 'GCAPassLive', 'GCAPassDead',
       'GCADrib', 'GCASh', 'GCAFld', 'GCADef'],
      dtype='object', length=119)

In [13]:
leeds_s1_all.drop([col for col in leeds_s1_all.columns if '_' in col], axis=1, inplace=True)

In [14]:
leeds_s1_all.shape

(22, 109)

In [15]:
duplicate_cols = leeds_s1_all.columns[leeds_s1_all.columns.duplicated()]
duplicate_cols
leeds_s1_all.drop(columns=duplicate_cols, inplace=True)

In [16]:
leeds_s1_all.shape

(22, 109)

In [17]:
#leeds_s1_all.to_csv('leeds-firstseason-allmerged.csv', index = False)

In [18]:
vals1 = join_all_columns("Villa", "1920", villa_s1, "GK")

In [19]:
def join_all_dfs_team(alldfslst):
    _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)
    ## duplicate columns will have endings like _x etc - removes these
    ## no columns had underscores to start
    _all.drop([col for col in _all.columns if '_' in col], axis=1, inplace=True)
    return _all

In [20]:
villa_s1_all = join_all_dfs_team(vals1)
#villa_s1_all.to_csv('villa-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


In [21]:
vals2 = join_all_columns("Newcastle", "1718", new_s1, "GK")
new_s2_all = join_all_dfs_team(vals2)
#new_s2_all.to_csv('newcastle-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


In [22]:
vals3 = join_all_columns("sheffutd", "1920", sheff_s1, "GK")
sheffutd_s1_all = join_all_dfs_team(vals3)
#sheffutd_s1_all.to_csv('sheffutd-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


In [23]:
vals4 = join_all_columns("huddersfield", "1718", hudds_s1, "GK")
hudds_s1_all = join_all_dfs_team(vals4)
#hudds_s1_all.to_csv('huddersfield-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


In [24]:
vals5 = join_all_columns("newcastle", "1718", new_s1, "GK")
new_s1_all = join_all_dfs_team(vals5)
new_s1_all.shape
#new_s1_all.to_csv('newcastle-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


(25, 109)

In [26]:
vals6 = join_all_columns("brighton", "1718", bri_s1, "GK")
bri_s1_all = join_all_dfs_team(vals6)
bri_s1_all.shape
bri_s1_all.to_csv('brighton-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)


In [27]:
vals7 = join_all_columns("wolves", "1819", wolves_s1, "GK")
wolves_s1_all = join_all_dfs_team(vals7)
#wolves_s1_all.to_csv('wolves-firstseason-allmerged.csv', index = False)

C:\Users\user\AppData\Local\Temp/ipykernel_21828/903767908.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'90s_x'} in the result is deprecated and will raise a MergeError in a future version.
  _all = reduce(lambda x, y: pd.merge(x, y, on = ["Player","Nation", "Age", "Pos"]), alldfslst)
